# BUILDING DEEP LEARNING APPLICATIONS WITH KERAS 2.0

## Preprocessing training data

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv")

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv")

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new pandas DataFrame objects from the scaled data
scaled_training_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

# Save scaled data dataframes to new CSV files
scaled_training_df.to_csv("sales_data_training_scaled.csv", index=False)
scaled_testing_df.to_csv("sales_data_testing_scaled.csv", index=False)

Note: total_earnings values were scaled by multiplying by 0.0000036968 and adding -0.115913


## Neural Network tensorflow

In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

#training_data_df = pd.read_csv(io.BytesIO(uploaded4["sales_data_training_scaled.csv"]))
training_data_df = pd.read_csv('sales_data_training_scaled.csv')

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

# Load the separate test data set
#test_data_df = pd.read_csv(io.BytesIO(uploaded5['sales_data_test_scaled.csv']))
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

# Save the model to disk
model.save("trained_model.h5")
print("Model saved to disk.")


Epoch 1/50
32/32 - 18s - loss: 0.0447 - 18s/epoch - 574ms/step
Epoch 2/50
32/32 - 0s - loss: 0.0072 - 95ms/epoch - 3ms/step
Epoch 3/50
32/32 - 0s - loss: 0.0017 - 72ms/epoch - 2ms/step
Epoch 4/50
32/32 - 0s - loss: 6.9449e-04 - 64ms/epoch - 2ms/step
Epoch 5/50
32/32 - 0s - loss: 4.4994e-04 - 66ms/epoch - 2ms/step
Epoch 6/50
32/32 - 0s - loss: 2.8514e-04 - 83ms/epoch - 3ms/step
Epoch 7/50
32/32 - 0s - loss: 2.2304e-04 - 84ms/epoch - 3ms/step
Epoch 8/50
32/32 - 0s - loss: 1.7738e-04 - 82ms/epoch - 3ms/step
Epoch 9/50
32/32 - 0s - loss: 1.3855e-04 - 68ms/epoch - 2ms/step
Epoch 10/50
32/32 - 0s - loss: 1.2102e-04 - 83ms/epoch - 3ms/step
Epoch 11/50
32/32 - 0s - loss: 1.0044e-04 - 65ms/epoch - 2ms/step
Epoch 12/50
32/32 - 0s - loss: 8.1284e-05 - 55ms/epoch - 2ms/step
Epoch 13/50
32/32 - 0s - loss: 7.2981e-05 - 54ms/epoch - 2ms/step
Epoch 14/50
32/32 - 0s - loss: 7.1308e-05 - 65ms/epoch - 2ms/step
Epoch 15/50
32/32 - 0s - loss: 7.8427e-05 - 68ms/epoch - 2ms/step
Epoch 16/50
32/32 - 0s - loss

## Load saved model


In [5]:
import pandas as pd
from keras.models import load_model

model = load_model('trained_model.h5')

X = pd.read_csv("proposed_new_product.csv").values
prediction = model.predict(X)

# Grab just the first element of the first prediction (since we only have one)
prediction = prediction[0][0]

# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = prediction + 0.1159
prediction = prediction / 0.0000036968

print("Earnings Prediction for Proposed Product - ${}".format(prediction))



Earnings Prediction for Proposed Product - $265639.96836314135


## Monitoring a Keras model with TensorBoard

In [11]:
# Load the tensorboard notebook extension
%load_ext tensorboard

In [23]:
import tensorflow as tf
import datetime, os
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Create a TensorBoard logger
# logger = keras.callbacks.TensorBoard(
#     log_dir='logs',
#     write_graph=True,
#     histogram_freq=5
# )
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
logger = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Train the model
model.fit(
    X,
    Y,
    epochs=200,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/200
32/32 - 1s - loss: 0.0209 - 796ms/epoch - 25ms/step
Epoch 2/200
32/32 - 0s - loss: 0.0024 - 369ms/epoch - 12ms/step
Epoch 3/200
32/32 - 0s - loss: 7.2140e-04 - 326ms/epoch - 10ms/step
Epoch 4/200
32/32 - 0s - loss: 3.7933e-04 - 308ms/epoch - 10ms/step
Epoch 5/200
32/32 - 0s - loss: 2.5808e-04 - 407ms/epoch - 13ms/step
Epoch 6/200
32/32 - 0s - loss: 1.8272e-04 - 237ms/epoch - 7ms/step
Epoch 7/200
32/32 - 0s - loss: 1.5674e-04 - 253ms/epoch - 8ms/step
Epoch 8/200
32/32 - 0s - loss: 1.3728e-04 - 354ms/epoch - 11ms/step
Epoch 9/200
32/32 - 0s - loss: 9.8432e-05 - 405ms/epoch - 13ms/step
Epoch 10/200
32/32 - 0s - loss: 1.0822e-04 - 408ms/epoch - 13ms/step
Epoch 11/200
32/32 - 0s - loss: 7.3283e-05 - 384ms/epoch - 12ms/step
Epoch 12/200
32/32 - 0s - loss: 5.8804e-05 - 427ms/epoch - 13ms/step
Epoch 13/200
32/32 - 0s - loss: 5.1776e-05 - 328ms/epoch - 10ms/step
Epoch 14/200
32/32 - 1s - loss: 5.0847e-05 - 565ms/epoch - 18ms/step
Epoch 15/200
32/32 - 0s - loss: 5.4506e-05 - 234ms/ep

Epoch 121/200
32/32 - 0s - loss: 5.4445e-05 - 337ms/epoch - 11ms/step
Epoch 122/200
32/32 - 0s - loss: 3.3325e-05 - 390ms/epoch - 12ms/step
Epoch 123/200
32/32 - 0s - loss: 3.3961e-05 - 310ms/epoch - 10ms/step
Epoch 124/200
32/32 - 0s - loss: 1.1229e-04 - 358ms/epoch - 11ms/step
Epoch 125/200
32/32 - 0s - loss: 4.6493e-05 - 330ms/epoch - 10ms/step
Epoch 126/200
32/32 - 0s - loss: 3.2383e-05 - 318ms/epoch - 10ms/step
Epoch 127/200
32/32 - 0s - loss: 2.2701e-05 - 292ms/epoch - 9ms/step
Epoch 128/200
32/32 - 0s - loss: 2.5792e-05 - 266ms/epoch - 8ms/step
Epoch 129/200
32/32 - 0s - loss: 3.2540e-05 - 314ms/epoch - 10ms/step
Epoch 130/200
32/32 - 0s - loss: 4.2039e-05 - 259ms/epoch - 8ms/step
Epoch 131/200
32/32 - 0s - loss: 2.6414e-05 - 244ms/epoch - 8ms/step
Epoch 132/200
32/32 - 0s - loss: 4.5813e-05 - 390ms/epoch - 12ms/step
Epoch 133/200
32/32 - 0s - loss: 5.8513e-05 - 277ms/epoch - 9ms/step
Epoch 134/200
32/32 - 0s - loss: 8.2822e-05 - 326ms/epoch - 10ms/step
Epoch 135/200
32/32 - 0s 

In [24]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 12588), started 0:12:33 ago. (Use '!kill 12588' to kill it.)

In [5]:
from platform import python_version
print(python_version())

3.8.12


In [6]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [1]:
print("&*")

&*
